In [1]:
from datetime import datetime
from django.utils.timezone import utc

from music.models import Album, Song, Artist, RecordCompany

In [12]:
Album.objects.bulk_create([
    Album(album_title="Humbug", artist="Arctic Monkeys", release_date=datetime(2009, 8, 19, 3, 59, 0, 99999, tzinfo=utc)),
    Album(album_title="Is This It", artist="The Strokes", release_date=datetime(2001, 7, 30, 3, 59, 0, 99999, tzinfo=utc)),
    Album(album_title="Happier Than Ever", artist="Billie Eilish", release_date=datetime(2021, 7, 30, 3, 59, 0, 99999, tzinfo=utc)),
    Album(album_title="Dreamland", artist="Glass Animals", release_date=datetime(2020, 8, 7, 3, 59, 0, 99999, tzinfo=utc)),
    Album(album_title="The Colour and the Shape", artist="Foo Fighters", release_date=datetime(1997, 5, 20, 3, 59, 0, 99999, tzinfo=utc))
])

[<Album: Humbug by Arctic Monkeys>,
 <Album: Is This It by The Strokes>,
 <Album: Happier Than Ever by Billie Eilish>,
 <Album: Dreamland by Glass Animals>,
 <Album: The Colour and the Shape by Foo Fighters>]

In [2]:
humbug = Album.objects.get(album_title="Humbug", artist="Arctic Monkeys")
is_this_it = Album.objects.get(album_title="Is This It", artist="The Strokes")
hte = Album.objects.get(album_title="Happier Than Ever", artist="Billie Eilish")
dreamland = Album.objects.get(album_title="Dreamland", artist="Glass Animals")
tcts = Album.objects.get(album_title="The Colour and the Shape", artist="Foo Fighters")

In [14]:
# Artists who composed the songs to be created
Artist.objects.bulk_create([
    Artist(name="Billie Eilish", birthday=datetime(2001, 12, 18, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Finneas O'Connell", birthday=datetime(1997, 7, 30, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Marina Diamandis", birthday=datetime(1985, 10, 10, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Dave Bayley", birthday=datetime(1989, 6, 7, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Brittany Hazzard", birthday=datetime(1990, 6, 14, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Paul Epworth", birthday=datetime(1974, 7, 25, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Julian Casablancas", birthday=datetime(1978, 8, 23, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Alex Turner", birthday=datetime(1986, 1, 6, 3, 59, 0, 99999, tzinfo=utc)),
    Artist(name="Arctic Monkeys"),
    Artist(name="Dave Grohl", birthday=datetime(1969, 1, 14, 3, 59, 0, 99999, tzinfo=utc)),
])

[<Artist: Billie Eilish>,
 <Artist: Finneas O'Connell>,
 <Artist: Marina Diamandis>,
 <Artist: Dave Bayley>,
 <Artist: Brittany Hazzard>,
 <Artist: Paul Epworth>,
 <Artist: Julian Casablancas>,
 <Artist: Alex Turner>,
 <Artist: Arctic Monkeys>,
 <Artist: Dave Grohl>]

In [3]:
artist_1 = Artist.objects.get(name="Billie Eilish")
artist_2 = Artist.objects.get(name="Finneas O'Connell")
artist_3 = Artist.objects.get(name="Marina Diamandis")
artist_4 = Artist.objects.get(name="Dave Bayley")
artist_5 = Artist.objects.get(name="Brittany Hazzard")
artist_6 = Artist.objects.get(name="Paul Epworth")
artist_7 = Artist.objects.get(name="Julian Casablancas")
artist_8 = Artist.objects.get(name="Alex Turner")
artist_9 = Artist.objects.get(name="Arctic Monkeys")
artist_10 = Artist.objects.get(name="Dave Grohl")

In [7]:
from music.choices import MUSIC_GENRES_CHOICES

song_1 = Song.objects.create(
    song_title="I Am Not A Robot",
    release_date=datetime(2010, 4, 23, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.pop_music
)
song_1.composers.set([artist_3])
song_2 = Song.objects.create(
    song_title="Everything I Wanted",
    release_date=datetime(2019, 11, 13, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.pop_music
)
song_2.composers.set([artist_1, artist_2])
song_3 = Song.objects.create(
    song_title="No Time To Die",
    release_date=datetime(2020, 2, 13, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.pop_music
)
song_3.composers.set([artist_1, artist_2])
song_4 = Song.objects.create(
    song_title="I Dont Wanna Talk (I Just Wanna Dance)",
    release_date=datetime(2020, 12, 13, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.pop_music
)
song_4.composers.set([artist_4])

# Humbug songs
song_5 = humbug.songs.create(
    song_title="My Propeller",
    release_date=datetime(2009, 8, 19, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.indie_rock
)
song_5.composers.set([artist_8, artist_9])
song_6 = humbug.songs.create(
    song_title="Crying Lightning",
    release_date=datetime(2009, 8, 7, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.indie_rock
)
song_6.composers.set([artist_8, artist_9])
song_7 = humbug.songs.create(
    song_title="Dance Little Liar",
    release_date=datetime(2009, 8, 19, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.indie_rock
)
song_7.composers.set([artist_8, artist_9])

# Is this it songs
song_9 = is_this_it.songs.create(
    song_title="Last Nite",
    release_date=datetime(2001, 10, 23, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.rock
)
song_9.composers.set([artist_7])
song_10 = is_this_it.songs.create(
    song_title="Take It Or Leave It",
    release_date=datetime(2001, 7, 30, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.rock
)
song_10.composers.set([artist_7])

# Happier Than Ever songs
song_11 = hte.songs.create(
    song_title="NDA",
    release_date=datetime(2021, 7, 9, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.electronic_music,
)
song_11.composers.set([artist_1, artist_2])
song_12 = hte.songs.create(
    song_title="Your Power",
    release_date=datetime(2021, 4, 29, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.pop_music
)
song_12.composers.set([artist_1, artist_2])
song_13 = hte.songs.create(
    song_title="Oxytocin",
    release_date=datetime(2021, 7, 30, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.electronic_music
)
song_13.composers.set([artist_1, artist_2])

# Dreamland songs
song_14 = dreamland.songs.create(
    song_title="Tangerine",
    release_date=datetime(2020, 8, 7, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.rock
)
song_14.composers.set([artist_4, artist_5, artist_6])
song_15 = dreamland.songs.create(
    song_title="Heat Waves",
    release_date=datetime(2020, 6, 29, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.pop_music
)
song_15.composers.set([artist_4])
song_16 = dreamland.songs.create(
    song_title="Helium",
    release_date=datetime(2020, 8, 7, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.rock
)
song_16.composers.set([artist_4])

# The Colour and the Shape songs
song_17 = tcts.songs.create(
    song_title="Everlong",
    release_date=datetime(1997, 8, 18, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.rock
)
song_17.composers.set([artist_10])
song_18 = tcts.songs.create(
    song_title="Monkey Wrench",
    release_date=datetime(1997, 4, 28, 3, 59, 0, 99999, tzinfo=utc),
    genre=MUSIC_GENRES_CHOICES.rock
)
song_18.composers.set([artist_10])

In [9]:
# Count example

from django.db.models import Count

Artist.objects.annotate(Count("songs_written")).filter(songs_written__count__gt=4)

<QuerySet [<Artist: Billie Eilish>, <Artist: Finneas O'Connell>]>

In [10]:
print(Artist.objects.annotate(Count("songs_written")).all().query)

SELECT "music_artist"."id", "music_artist"."name", "music_artist"."birthday", COUNT("music_song_composers"."song_id") AS "songs_written__count" FROM "music_artist" LEFT OUTER JOIN "music_song_composers" ON ("music_artist"."id" = "music_song_composers"."artist_id") GROUP BY "music_artist"."id"


In [11]:
# Using F with annotations

from django.db.models import F, Value
from django.db.models.functions import Concat

Album.objects.annotate(album_label=Concat(F("album_title"), Value(" by "), F("artist"))).all().first().__dict__

{'_state': <django.db.models.base.ModelState at 0x7f4101fc15b0>,
 'id': 28,
 'album_title': 'Humbug',
 'artist': 'Arctic Monkeys',
 'record_company_id': None,
 'release_date': None,
 'album_label': 'Humbug by Arctic Monkeys'}

In [14]:
# Annotation with filters

from django.db.models import Q

Album.objects.annotate(
    electronic_songs_count=Count("songs", filter=Q(songs__genre=MUSIC_GENRES_CHOICES.electronic_music))
).values_list("album_title", "electronic_songs_count")

<QuerySet [('Dreamland', 0), ('Is This It', 0), ('Humbug', 0), ('The Colour and the Shape', 0), ('Happier Than Ever', 2)]>

In [ ]:
# Using FilteredRelation with annotations

In [ ]:
# Using SubQuery with annotations